In [1]:
import pandas as pd

data_path = "./data/"

In [2]:
train_df = pd.read_csv(data_path + "data_training_full.csv")
train_df.head()

,Unnamed: 0,Argument ID,Conclusion,Stance,Premise,text,category,Self-direction: thought,Self-direction: action,Stimulation,...,Tradition,Conformity: rules,Conformity: interpersonal,Humility,Benevolence: caring,Benevolence: dependability,Universalism: concern,Universalism: nature,Universalism: tolerance,Universalism: objectivity
0,4568,A30388,We should adopt an austerity regime,against,we should not support the austerity regime bec...,we should not support the austerity regime bec...,"['Security: personal', 'Benevolence: caring', ...",0,0,0,...,0,0,0,0,1,0,1,0,0,1
1,1142,A19198,We should legalize cannabis,in favor of,in many cases cannabis has wonderful health re...,in many cases cannabis has wonderful health re...,"['Self-direction: action', 'Security: personal...",0,1,0,...,0,0,0,0,1,0,0,1,0,0
2,743,A18079,We should ban cosmetic surgery,in favor of,the long term effects of new procedures have n...,the long term effects of new procedures have n...,"['Security: personal', 'Benevolence: caring', ...",0,0,0,...,0,0,0,0,1,1,0,0,0,0
3,4824,E01075,Devices with batteries should be designed so t...,in favor of,Having to replace an entire mobile phone (or t...,Having to replace an entire mobile phone (or t...,"['Security: personal', 'Universalism: nature']",0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,2168,A22268,We should legalize cannabis,in favor of,we should legalize cannabis because it is harm...,we should legalize cannabis because it is harm...,"['Self-direction: action', 'Hedonism', 'Humili...",0,1,0,...,0,0,0,1,0,0,0,0,0,0


In [3]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# 设定随机种子以便复现性
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

# 读取数据并预处理
premises = train_df['Premise']  # 你的前提文本数据
conclusions = train_df['Conclusion']  # 你的结论文本数据
stances = train_df['Stance']  # 0代表against，1代表infavorof，以此类推

print(premises.head())
print(conclusions.head())
print(stances.head())

stance_mapping = {"against": 0, "in favor of": 1}
labels = [stance_mapping[stance] for stance in stances]
print(labels)

c:\Users\96938\anaconda3\envs\pytorch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

In [ ]:
# 使用RoBERTa tokenizer来编码文本
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
encoded_inputs = tokenizer(list( conclusions+ "." + premises), padding=True, truncation=True, return_tensors="pt", max_length=100)

input_ids = encoded_inputs["input_ids"]
attention_mask = encoded_inputs["attention_mask"]


In [ ]:
# 划分训练集和测试集
input_ids_train, input_ids_test, attention_mask_train, attention_mask_test, labels_train, labels_test = train_test_split(
    input_ids, attention_mask, labels, test_size=0.2, random_state=seed
)

print(input_ids_train.shape)
print(input_ids_test.shape)
print(attention_mask_train.shape)
print(attention_mask_test.shape)

torch.Size([5500, 100])
torch.Size([1376, 100])
torch.Size([5500, 100])
torch.Size([1376, 100])


In [ ]:
# 加载预训练的RoBERTa模型
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)

# 设置GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [ ]:
# 创建数据加载器
batch_size = 16
train_dataset = TensorDataset(input_ids_train, attention_mask_train, torch.tensor(labels_train))
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# 定义优化器和学习率
optimizer = AdamW(model.parameters(), lr=1e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# 训练模型
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        input_ids_batch, attention_mask_batch, labels_batch = batch
        input_ids_batch, attention_mask_batch, labels_batch = input_ids_batch.to(device), attention_mask_batch.to(device), labels_batch.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids_batch, attention_mask=attention_mask_batch, labels=labels_batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1} Loss: {total_loss / len(train_loader)}")



Epoch 1: 100%|██████████| 344/344 [01:36<00:00,  3.56it/s]


Epoch 1 Loss: 0.45203227079798314


Epoch 2: 100%|██████████| 344/344 [01:35<00:00,  3.58it/s]


Epoch 2 Loss: 0.1920798634298059


Epoch 3: 100%|██████████| 344/344 [01:35<00:00,  3.59it/s]

Epoch 3 Loss: 0.1217518970681636


In [ ]:
# 在测试集上进行评估
model.eval()
with torch.no_grad():
    input_ids_test, attention_mask_test, labels_test = input_ids_test.to(device), attention_mask_test.to(device), torch.tensor(labels_test).to(device)
    outputs = model(input_ids=input_ids_test, attention_mask=attention_mask_test)
    logits = outputs.logits
    predicted_labels = torch.argmax(logits, dim=1).cpu().numpy()


accuracy = accuracy_score(labels_test.cpu().numpy(), predicted_labels)
print(f"Accuracy on Test Set: {accuracy * 100:.2f}%")

Accuracy on Test Set: 92.66%


In [ ]:
# !pip install toolbox

In [ ]:
# from toolbox.data_preprocessing import create_data_file

In [ ]:
data_path = "./data/appendix/"

test_data = pd.read_csv(data_path + "comments-level1.tsv", delimiter='\t', encoding='utf-8')
test_data.head()

,Argument ID,Worker ID,Value,Comment
0,A27031,W034,Have life accepted as is,Thjis is the only thing that comes close to fi...
1,A27031,W034,-,This is a really weak argument and nothing tru...
2,A27032,W034,Be just,This sort of fits
3,A27034,W034,Be helpful,This is ultimately what they're wanting to do ...
4,A27035,W034,-,Because it is good to protect the vulnerable


In [ ]:
# 预处理测试数据
def preprocess_text(text):
    # 使用tokenizer编码文本
    # tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
    encoded_inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt", max_length=100)
    input_ids = encoded_inputs["input_ids"]
    attention_mask = encoded_inputs["attention_mask"]
    return input_ids, attention_mask


In [ ]:
# 预测函数
def predict(text):
    input_ids, attention_mask = preprocess_text(text)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    with torch.no_grad():
        input_ids, attention_mask = input_ids.to(device), attention_mask.to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predicted_labels = torch.argmax(logits, dim=1).cpu().numpy()
    return predicted_labels

In [ ]:
topic=['Self-direction:thought','Self-direction:action','Stimulation','Hedonism','Achievement','Power:dominance','Power:resources','Face','Security:personal','Security:societal','Tradition','Conformity:rules','Conformity: interpersonal','Humility','Benevolence:caring','Benevolence:dependability','Universalism:concern','Universalism:nature','Universalism:tolerance']
print(len(topic))

19


In [ ]:
describition=['Freedom to cultivate one\'s own ideas and abilities.Personal motivation: It is important to be creative, forming own opinions, be unique, have original ideas, learn things for oneself and improve own abilities.The focus of this value is on developing own ideas, wanting to know more and discovering.',
'Freedom to determine one\'s own actions.Personal motivation: It is important to make own decisions about life, being independent and having the freedom to choose.The focus of this value in contrast to the \"self-direction: thought\" aspect is to determine an action, rather than a conviction or thought.',
'Excitement, novelty, and change.Personal motivation: Always looking for something new to do, doing something exciting, seeking out new experiences, innovating, being bold, seeking adventures and initiating change.In contrast to \"hedonism\", this value focuses on the novelty and risk aspects of behaviours and thoughts. It is seeking out everything that stimulates the senses.',
'Pleasure and sensuous gratification.Personal motivation: Having a good time, enjoying life\’s pleasures and taking advantage of opportunities to have fun.',
'Success according to social standards.Personal motivation: Being ambitious, successful and being admired for achievements and skills. Demonstrating competence according to social standards or in competition.Important dimension of achievement is that it is perceived within the social standards, and according to the rules of engagement, unlike as for the value of power. Moreover, unlike the value of power, Achievement focuses on performance and not on resource matters.',
'Power through exercising control over people.Personal motivation: Want people to follow you, being the most influential compared to others, be the one to determine directions.',
'Power through control of material and social resources.Personal motivation: Having lots of money for the power it brings, being wealthy, and pursue social status.This value is not automatically present when terms like budgets, costs, or growth appear in a sentence. Such terms refer to the value only if they make up an essential part of a sentence and are expressed as an important part of decision making with an underlying motivation or justification. For example, when someone is writing about the \"huge\", \"large\", \"out of control\" costs, or in any other way mentioned as important argument. Finally, in contrast to the value of \"achievement\", it only focuses on resource matters not on performance.',
'Security and power through maintaining one\’s public image and avoiding humiliation.Personal motivation: Does not want to be shamed by others, protecting public image, being treated with respect, honour, and dignity.',
'Safety in one\'s immediate environment.Personal motivation: Avoid dangerous situations, value personal security and safety, live in a secure environment, have a secured income, being healthy.Similar to the difference between \"benevolence\" and \"universalism: concern\", the decisive difference between \"security: personal\" and \"security: societal\" is to whom the value refers. In case of individuals, family and friends, it is personal, e.g., if someone writes in first-person about their own health. In contrast, when it is applied to any group or society as a whole, it is societal. Thus, \"security: personal\" will most likely not appear often in the text only if speaking of individual experiences.',
'Safety and stability in the wider society.Personal motivation: Country should protect itself against all threats, state should be strong, order and stability in society are important, including economic stability (employment, no recession). Importantly, the value refers not only to a society as a whole but also to socially defined groups like women, parents, etc. within a society. In contrast to the value of \"universalism: concern\", the value emphasises protection more from a motivation of \"preventing\", \"averting\", \"ending\" dangers or threats and \"preserving\" security and stability.',
'Maintaining and preserving cultural, family, or religious traditions.Personal motivation: Maintain traditional beliefs and values, follow the family or religious customs, valuing traditional practices of one\’s culture.',
'Compliance with rules, laws, and formal obligations.Personal motivation: Should follow authorities, follow rules even if others are not watching, obey all laws.',
'Avoidance of upsetting or harming other people.Personal motivation: Avoid upsetting or annoying others, being tactful to others, showing courtesy, being polite, resisting temptation, respecting elders.',
'Recognizing one\'s insignificance in the larger scheme of things.Personal motivation: Try not to draw attention, be humble and satisfied with the situation, not asking for more.',
'Devotion to the welfare of in-group members.Personal motivation: Help and care about close ones, be responsive to family and friends. Actively helping and taking care of someone close.A key difference between \"benevolence: caring\" and \"benevolence: dependability\" is that the caring dimension is active about doing something, while the dependability is more about the perception of being there for someone, being trustworthy etc. Furthermore, caring here is not generic or universal, it is limited to specific persons or a delineated in-group within a person\’s immediate environment. This last aspect distinguishes \"benevolence: caring\" from \"universalism: concern\". Similar to the two sides of security, the decisive difference between \"benevolence: caring\" and \"universalism: concern\" is to whom the value refers. In case of individuals, family, and friends, it is \"benevolence: caring\". In contrast, when it is applied to any social group or people in general, it is \"universalism: concern\". Thus, \"benevolence: caring" will most likely not appear often in the text only if speaking of individual experiences.',
'Being a reliable and trustworthy member of the in-group.Personal motivation: Be loyal to close ones, be dependable and trustworthy, especially to close ones (in-group). Be seen as reliable, others should have confidence in you helping close ones.A key difference between \"benevolence\" and \"universalism\" is that \"benevolence\" is primarily targeted towards close ones, and not towards strangers. In line with \"security: personal\", close ones are family and friends, not the larger public.',
'Commitment to equality, justice, and protection for all people.Personal motivation: Protecting the weak and vulnerable, care about equal opportunities, treat everyone justly.In contrast to the value of \"security: societal\", protection and caring also goes beyond society boundaries. It is more generic referring to all kinds of people/groups. Moreover, \"universalism: concern\" focuses more on caring, protecting, promoting well-being, especially of vulnerable people from a motivation of \"empathy\", \"helping\" or \"a universal justice perspective\", whereas \"security: societal\" focuses on protecting, promoting well-being more from a motivation of \"preventing\", \"averting\", \"ending\" dangers or threats and \"preserving\" security and stability.',
'Preservation of the natural environment.Personal motivation: Care about nature for nature\'s sake, protect the environment against pollution, destruction and other threats.',
'Acceptance and understanding of those who are different from oneself.Personal motivation: Care about peace and harmony, listen to people with other views, understand even those one disagrees with.The key point here is that it is peace for harmony\'s sake and not for the protection of the weak, which is covered in \"universalism: concern\" already.']
print(len(describition))

19


In [ ]:
# 对测试数据进行预测
for index, row in test_data.iterrows():
   print(f"Text: {row['Comment']}")
   for t in range(19):
        text = row['Comment'] + topic[t] + describition[t]
        predicted_label = predict(text)
        print(f"Topic: {topic[t]}")
        print(f"Predicted Label: {predicted_label[0]}")

Streaming output truncated to the last 5000 lines.
Topic: Benevolence:dependability
Predicted Label: 1
Topic: Universalism:concern
Predicted Label: 0
Topic: Universalism:nature
Predicted Label: 0
Topic: Universalism:tolerance
Predicted Label: 1
Text: it is good to let things play out
Topic: Self-direction:thought
Predicted Label: 1
Topic: Self-direction:action
Predicted Label: 1
Topic: Stimulation
Predicted Label: 1
Topic: Hedonism
Predicted Label: 1
Topic: Achievement
Predicted Label: 1
Topic: Power:dominance
Predicted Label: 1
Topic: Power:resources
Predicted Label: 0
Topic: Face
Predicted Label: 1
Topic: Security:personal
Predicted Label: 1
Topic: Security:societal
Predicted Label: 1
Topic: Tradition
Predicted Label: 1
Topic: Conformity:rules
Predicted Label: 1
Topic: Conformity: interpersonal
Predicted Label: 1
Topic: Humility
Predicted Label: 1
Topic: Benevolence:caring
Predicted Label: 1
Topic: Benevolence:dependability
Predicted Label: 1
Topic: Universalism:concern
Predicted Lab